# DATA WHAREHOUSE (ETL)

## Import

In [1]:
# Traitement et manipulation des données
import pandas as pd
import os
import re

# connect to S3
import boto3

# Connexion base de données
from sqlalchemy import create_engine

# Gestion des variables d'environnement
from dotenv import load_dotenv
load_dotenv()


True

## Init

In [2]:
PASSWORD = os.getenv("PASSWORD")
HOSTNAME = os.getenv("HOSTNAME") 
DBNAME = os.getenv("DBNAME")

# Configuration S3
bucket = os.getenv("AWS_BUCKET_NAME")
folder = os.getenv("AWS_BUCKET_FOLDER")
region = os.getenv("AWS_REGION")
access_key = os.getenv("aws_access_key_id")
secret_key = os.getenv("aws_secret_access_key")


In [3]:

def extract_from_s3(filename):
    """
    Récupère un fichier CSV depuis un bucket S3 et retourne un DataFrame.
    """
    if not all([bucket, folder, access_key, secret_key]):
        raise ValueError("❌ Une ou plusieurs variables d'environnement S3 sont manquantes.")

    # Initialisation du client S3
    s3 = boto3.client(
        "s3",
        region_name=region,
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key
    )

    # Clé S3 complète
    key = f"{folder}/{filename}"

    try:
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(obj['Body'], encoding='utf-8-sig')
        print(f"[✅] Fichier '{filename}' extrait depuis s3://{bucket}/{key}")
        return df
    except Exception as e:
        print(f"[❌] Erreur lors de l'extraction de '{filename}' : {e}")
        return None





## Pipeline ETL City data

### Extract

In [4]:
df_extract_city = extract_from_s3('df_city.csv')
if df_extract_city is not None:
    display(df_extract_city.head())

[✅] Fichier 'df_city.csv' extrait depuis s3://licorne2lc-kayak/Kayack/df_city.csv


,City_id,City,Latitude,Longitude
0,FR000001,Mont Saint Michel,48.635954,-1.511460
1,FR000002,St Malo,48.649518,-2.026041
2,FR000003,Bayeux,49.276462,-0.702474
3,FR000004,Le Havre,49.493898,0.107973
4,FR000005,Rouen,49.440459,1.093966


### Transform

In [5]:
df_trans_city = df_extract_city.copy()

# Renommage des colonnes pour éviter les conflits
df_trans_city.rename(columns={"Latitude": "City_lat",
                                "Longitude": "City_long"},
                                inplace=True)

# export CSV
df_trans_city.to_csv("D:/jedha/full_stack/projet/Scraping_Kayak/df_trans_city.csv", index=False, encoding="utf-8-sig")
print("[✅] df_trans_city exporté.")

df_trans_city.head()



[✅] df_trans_city exporté.


,City_id,City,City_lat,City_long
0,FR000001,Mont Saint Michel,48.635954,-1.511460
1,FR000002,St Malo,48.649518,-2.026041
2,FR000003,Bayeux,49.276462,-0.702474
3,FR000004,Le Havre,49.493898,0.107973
4,FR000005,Rouen,49.440459,1.093966


### Load

In [6]:
engine = create_engine(f"postgresql+psycopg2://licorne2lc:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True)

df_trans_city.to_sql("df_trans_city", engine, if_exists="replace", index=False)

2025-06-16 08:11:05,370 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-16 08:11:05,370 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:05,381 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-16 08:11:05,383 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:05,393 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-16 08:11:05,394 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:05,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-16 08:11:05,408 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

35

## Pipeline ETL hotels data

### Extract

In [7]:
df_extract_hotels = extract_from_s3('df_hotels.csv')
if df_extract_hotels is not None:
    display(df_extract_hotels.head())

[✅] Fichier 'df_hotels.csv' extrait depuis s3://licorne2lc-kayak/Kayack/df_hotels.csv


,City_id,city,nom,url,note,latlong,adresse,points_forts
0,FR000001,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,"8,2","48.6175872716489,-1.51039615273476","La Caserne, 50170 Le Mont-Saint-Michel, France","['Ses points forts', 'Parking', 'Petit-déjeune..."
1,FR000001,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,"8,0","48.61470048629041,-1.5096169710159302","La Caserne, 50170 Le Mont-Saint-Michel, France","['Ses points forts', 'LA SALICORNE', 'Restaura..."
2,FR000001,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"8,3","48.61424652959294,-1.510545015335083","La Caserne, 50170 Le Mont-Saint-Michel, France","['Ses points forts', 'Parking', 'Petit-déjeune..."
3,FR000001,Mont Saint Michel,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,"8,2","48.61538141368341,-1.510709971189499","Route du Mont Saint Michel, 50170 Le Mont-Sain...","['Ses points forts', 'LA SALICORNE', 'LE PRE S..."
4,FR000001,Mont Saint Michel,Auberge Saint Pierre,https://www.booking.com/hotel/fr/auberge-saint...,"8,3","48.63568797869143,-1.5098825097084045","Grande Rue, 50170 Le Mont-Saint-Michel, France","['Ses points forts', 'Restaurant #1']"


### Transform

In [8]:
df_trans_hotels = df_extract_hotels.copy()

# Harmonisation de la casse des colonnes
df_trans_hotels.columns = [col.strip().lower().replace(" ", "_").capitalize() for col in df_trans_hotels.columns]

# Séparation de la colonne latlong en deux colonnes numériques
df_trans_hotels[["Hotel_lat", "Hotel_long"]] = (
    df_trans_hotels["Latlong"].str.split(",", expand=True).astype(float)
)

# Suppression de la colonne latlong
df_trans_hotels.drop(columns=["Latlong"], inplace=True)

# Nettoyage de la colonne points_forts avec regex
def clean_points_forts_regex(text):
    # Supprimer "Ses points forts" avec ou sans majuscules
    text = re.sub(r"'?Ses points forts'?,?\s*", "", text, flags=re.IGNORECASE)

    # Supprimer les crochets restants
    text = re.sub(r"^\[|\]$", "", text.strip())

    # Split, strip et nettoyage
    items = [re.sub(r"^['\"\s,]+|['\"\s,]+$", "", part) for part in text.split(",")]

    # Transformation des "#chiffre" → "chiffre*"
    items = [re.sub(r"#(\d+)", r"\1*", item) for item in items if item]

    return items

# Application du nettoyage
df_trans_hotels["Points_forts"] = df_trans_hotels["Points_forts"].apply(clean_points_forts_regex)

# export CSV
df_trans_hotels.to_csv("D:/jedha/full_stack/projet/Scraping_Kayak/df_trans_hotels.csv", index=False, encoding="utf-8-sig")
print("[✅] df_trans_hotels exporté.")

df_trans_hotels


[✅] df_trans_hotels exporté.


,City_id,City,Nom,Url,Note,Adresse,Points_forts,Hotel_lat,Hotel_long
0,FR000001,Mont Saint Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,"8,2","La Caserne, 50170 Le Mont-Saint-Michel, France","[Parking, Petit-déjeuner disponible, Vue, Anim...",48.617587,-1.510396
1,FR000001,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,"8,0","La Caserne, 50170 Le Mont-Saint-Michel, France","[LA SALICORNE, Restaurant 2*]",48.614700,-1.509617
2,FR000001,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"8,3","La Caserne, 50170 Le Mont-Saint-Michel, France","[Parking, Petit-déjeuner disponible, Animaux a...",48.614247,-1.510545
3,FR000001,Mont Saint Michel,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,"8,2","Route du Mont Saint Michel, 50170 Le Mont-Sain...","[LA SALICORNE, LE PRE SALE]",48.615381,-1.510710
4,FR000001,Mont Saint Michel,Auberge Saint Pierre,https://www.booking.com/hotel/fr/auberge-saint...,"8,3","Grande Rue, 50170 Le Mont-Saint-Michel, France",[Restaurant 1*],48.635688,-1.509883
...,...,...,...,...,...,...,...,...,...
868,FR000035,La Rochelle,Vintage calme sur jardin en centre historique,https://www.booking.com/hotel/fr/vintage-calme...,"9,6","11 Rue Buffeterie, 17000 La Rochelle, France","[L’hébergement entier est pour vous, Vue, Équi...",46.162419,-1.148105
869,FR000035,La Rochelle,Tour de Nesle La Rochelle Vieux Port 3 etoiles,https://www.booking.com/hotel/fr/de-la-tour-de...,"8,7","2, Quai Louis Durand, 17000 La Rochelle, France","[Petit-déjeuner disponible, Vue, Animaux admis...",46.158097,-1.150362
870,FR000035,La Rochelle,Hôtel Les Brises,https://www.booking.com/hotel/fr/hotellesbrise...,"8,6","Chemin de la digue Richelieu, 17000 La Rochell...",[],46.153458,-1.174386
871,FR000035,La Rochelle,"Appartement Charlie, Wifi, Charme, Centre Hist...",https://www.booking.com/hotel/fr/appartement-c...,"8,3","1 Rue Chaudrier, 17000 La Rochelle, France","[L’hébergement entier est pour vous, Vue, Équi...",46.160301,-1.153571


### Load

In [9]:
engine = create_engine(f"postgresql+psycopg2://licorne2lc:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True)

df_trans_hotels.to_sql("df_trans_hotels", engine, if_exists="replace", index=False)

2025-06-16 08:11:05,919 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-16 08:11:05,920 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:05,931 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-16 08:11:05,931 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:05,943 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-16 08:11:05,943 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:05,955 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-16 08:11:05,957 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

873

## Pipeline ETL weather data

### Extract

In [10]:
df_extract_meteo_forecast = extract_from_s3('df_meteo_forecast.csv')
if df_extract_meteo_forecast is not None:
    display(df_extract_meteo_forecast.head())

[✅] Fichier 'df_meteo_forecast.csv' extrait depuis s3://licorne2lc-kayak/Kayack/df_meteo_forecast.csv


,City_id,City,Latitude,Longitude,Date,Température max (°C),Vent max (m/s),Force vent,Angle vent (°),Indice UV,Icon,Emoji,Description
0,FR000001,Mont Saint Michel,48.635954,-1.51146,2025-06-16,20,5,3,328,8.28,01d,☀️,Ciel dégagé (jour)
1,FR000001,Mont Saint Michel,48.635954,-1.51146,2025-06-17,22,5,3,324,8.40,01d,☀️,Ciel dégagé (jour)
2,FR000001,Mont Saint Michel,48.635954,-1.51146,2025-06-18,26,6,4,44,8.42,02d,🌤️,Peu nuageux (jour)
3,FR000001,Mont Saint Michel,48.635954,-1.51146,2025-06-19,29,5,3,75,7.67,02d,🌤️,Peu nuageux (jour)
4,FR000001,Mont Saint Michel,48.635954,-1.51146,2025-06-20,30,5,3,101,8.07,04d,☁️,Couvert


### Transform

In [11]:
df_trans_weather = df_extract_meteo_forecast.copy()

# Nettoyage de la colonne "Description" : suppression du contenu entre parenthèses
def remove_parentheses_content(text):
    return re.sub(r"\s*\([^)]*\)", "", str(text)).strip()

df_trans_weather["Description"] = df_trans_weather["Description"].apply(remove_parentheses_content)

# Renommage des colonnes pour éviter les conflits
df_trans_weather.rename(columns={"Latitude": "City_lat",
                              "Longitude": "City_long",
                              "Description":"Weather_description"},
                              inplace=True)

# export CSV
df_trans_weather.to_csv("D:/jedha/full_stack/projet/Scraping_Kayak/df_trans_weather.csv", index=False, encoding="utf-8-sig")
print("[✅] df_trans_weather exporté.")

df_trans_weather.head(1)

[✅] df_trans_weather exporté.


,City_id,City,City_lat,City_long,Date,Température max (°C),Vent max (m/s),Force vent,Angle vent (°),Indice UV,Icon,Emoji,Weather_description
0,FR000001,Mont Saint Michel,48.635954,-1.51146,2025-06-16,20,5,3,328,8.28,01d,☀️,Ciel dégagé


### Load

In [12]:
engine = create_engine(f"postgresql+psycopg2://licorne2lc:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True)

df_trans_weather.to_sql("df_trans_weather", engine, if_exists="replace", index=False)

2025-06-16 08:11:06,367 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-16 08:11:06,367 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:06,379 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-16 08:11:06,379 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:06,391 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-16 08:11:06,391 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:06,404 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-16 08:11:06,406 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

280

## Database (fusion+ export SQL)

In [13]:
def merge_dataframes_on_city_id(df1, df2):
    # Supprime les colonnes du df2 déjà présentes dans df1 (hors City_id)
    df2_clean = df2.drop(columns=[col for col in df2.columns if col in df1.columns and col != "City_id"])
    return pd.merge(df1, df2_clean, on="City_id", how="inner")

# Fusion city + hotels
merged_city_hotels = merge_dataframes_on_city_id(df_trans_city, df_trans_hotels)

# Fusion avec météo
df_final = merge_dataframes_on_city_id(merged_city_hotels, df_trans_weather)

# ✅ Affichage des dimensions
print(f"✅ La fusion des dataframes contient :\n{df_final.shape[0]} lignes\n{df_final.shape[1]} colonnes")

# export CSV
df_final.to_csv("D:/jedha/full_stack/projet/Scraping_Kayak/database.csv", index=False, encoding="utf-8-sig")
print("[✅] database exportée.")

# Initialisation client S3
s3 = boto3.client(
    "s3",
    region_name=region,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key)

print("✅ Database envoyés avec succès vers S3.")

# Aperçu
display(df_final.head(2))


✅ La fusion des dataframes contient :
6984 lignes
20 colonnes
[✅] database exportée.
✅ Database envoyés avec succès vers S3.


,City_id,City,City_lat,City_long,Nom,Url,Note,Adresse,Points_forts,Hotel_lat,Hotel_long,Date,Température max (°C),Vent max (m/s),Force vent,Angle vent (°),Indice UV,Icon,Emoji,Weather_description
0,FR000001,Mont Saint Michel,48.635954,-1.51146,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,"8,2","La Caserne, 50170 Le Mont-Saint-Michel, France","[Parking, Petit-déjeuner disponible, Vue, Anim...",48.617587,-1.510396,2025-06-16,20,5,3,328,8.28,01d,☀️,Ciel dégagé
1,FR000001,Mont Saint Michel,48.635954,-1.51146,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,"8,2","La Caserne, 50170 Le Mont-Saint-Michel, France","[Parking, Petit-déjeuner disponible, Vue, Anim...",48.617587,-1.510396,2025-06-17,22,5,3,324,8.40,01d,☀️,Ciel dégagé


In [14]:
engine = create_engine(f"postgresql+psycopg2://licorne2lc:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True)

df_final.to_sql("database", engine, if_exists="replace", index=False)

2025-06-16 08:11:06,761 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-16 08:11:06,762 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:06,773 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-16 08:11:06,774 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:06,784 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-16 08:11:06,785 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-16 08:11:06,796 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-16 08:11:06,800 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

984